<a href="https://colab.research.google.com/github/jmcarbo/databases/blob/master/database_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pràctica 2: Disseny de Bases de Dades en Medicina

Assignatura: **Informàtica Biomèdica**<br/>
Curs: **2020-2021**<br/>
Professor: *Joan Marc Carbó* (joan.carbo@upf.edu)

## Com fer servir aquest quadern de pràctiques electrònic?



Aquest és un quadern de pràctiques interactiu en el que trobareu dos tipus de paràgrafs (**cel·les**): els paràgafs de tipus **text** i els paràgrafs de tipus **programes executables** (*codi*), que contenen instruccions per l'ordinador i estan escrits en el llenguatge de programació *Python*.

Els paràgrafs que contenen **programes** (*codi*) es distingeixen perquè tenen fons gris clar i quan situem el cursor sobre els corchets ```[ ]``` de la part superior exquerra es converteix en una icona amb un triangle que podem clicar i fer que s'executi el codi que hi emmagatzemat. Si apareix un número entre els corchets, com per exemple ```[3]``` voldrà dir que el paràgraf de codi s'ha executat com a mínim un cop durant la sessió actual. El número pròpiament dit indica l'ordre en que s'ha executat respecte de la resta de paràgrafs.

Cal tenir cura perquè tots els **paràgrafs són editables** i si escribim a sobre modificarem el seu contingut. En el cas dels paràgrafs de text cal clicar dos cops per poder-los editar i clicar fora del paràgraf per finalitzar l'edició. En el cas dels paràgrafs de codi es poden editar directament només posant el cursor sobre el paràgraf.

A la part superior esquerra d'aquesta plana trobareu dos botons amb el text **"+ Code"** i **"+ Text"** que us permetran afegir nous paràgrafs de *text* o *codi* just a sota del paràgraf on tingueu el cursor.

Podeu guardar-vos una còpia d'aquest quadern de pràctiques anant al menú de la capçalera de la pàgina i seleccionant "File" i a continuació en el desplegable escollir **"Save a copy in drive"** (us guradarà una còpia en el vostre Google Drive) o bé seleccionar l'opció **"Download .ipynb"** i us guardarà una còpia en el vostre ordinador local. Si escolliu l'opció Google Drive podreu anar salvant a demanda una còpia actualitzada només sel·lecionat l'opció **"save"**.

### Exercici 1:

> Proveu d'afegir un paràgraf pde text sota d'aquest mateix paràgraf i escriu-hi el teu NIA i el teu nom i cognoms



## Objectiu de la pràctica

L'objectiu de la pràctica de disseny de bases de dades mèdiques és experimentar amb la **captura, emmagatzemament, recuperació i anàlisi de dades mèdiques** i què millor que fer servir dades referents a la pandèmia COVID-19.

## Origen de les dades que analitzarem

Les **dades** que farem servir són les que recull diàriament *The Center for Systems Science and Engineering* (**CSSE**) de la John Hopkins University i que estan lliurament disponibles a l'adreça https://github.com/CSSEGISandData/COVID-19

## Com obtenim les dades?

Ha arribat el moment de començar a explotar les dades però primer les hem de tenir a ma i per això haurem de donar-li instruccions a l'ordinador de com fer-ho.

El primer que haurem de fer es dir-li quines biblioteques de programes volem fer servir. Una biblioteca de programa és un conjunt d'instruccions que han escrit prèviament altres persones i que ens faciliten a nosaltres la feina perquè contenen conjunts d'instruccions que fan accions complexes que, d'altra manera, hauriem de escriure nosaltres desde zero.

Com a primer pas executarem el paràgraf de codi que ve a continuació i que li indica a l'ordinador quines biblioteques volem fer servir. La **instrucció** `import` indica que incorpori una biblioteca de funcions.

Premeu doncs sobre el triangle d'execució del paràgraf següent per executar el codi. Heu d'observar que la icona comença a donar voltes i que finalment apareix un número dins dels corchets o bé el número que ja hi havia s'incrementa. Això ens indicarà que l'ordinador ja ha processat les instruccions que li hem donat.

In [4]:
import pandas as pd
import numpy
import altair as alt
import numpy as np
import glob
import os

A continuació descarreguem les dades originals amb l'instrucció `git`. Fem servir aquesta instrucció perquè les dades COVID-19 que farem servir estan emmagatzemades en unes carpetes especials que permeten versionar la informació i controlar els canvis que es van fen. Axiò és especialment interessant quan fem recerca ja que permet garantir que la informació no es malmet per errors informàtics i, a més, ens permet acreditar qui ha fet què.

Executeu doncs el paràgraf següent, trigarà uns pocs minuts donat que ha de descarregar un bon grapat d'arxius. Sabreu que l'execució ha finalitzat quan l'icona d'execució deixi de donar voltes.

In [5]:
!git clone https://github.com/CSSEGISandData/COVID-19.git

fatal: destination path 'COVID-19' already exists and is not an empty directory.


Per assegurar-nos que s'han descarregat correctament els arxius, li direm ara a l'ordinador que ens digui quins arxius hi ha ara a la nova carpeta que hem creat que s'anomena "`csse_covid_19_daily_reports`" i per això fem servir l'instrucció `ls`

Executeu doncs el paràgraf que ve a continuació. A partir d'ara ja no us indicarem que un paràgraf amb codi s'ha d'executar per continuar avançant en la pràctica.



In [6]:
!ls COVID-19/csse_covid_19_data/csse_covid_19_daily_reports

01-01-2021.csv	03-23-2020.csv	06-03-2020.csv	08-14-2020.csv	10-25-2020.csv
01-02-2021.csv	03-24-2020.csv	06-04-2020.csv	08-15-2020.csv	10-26-2020.csv
01-03-2021.csv	03-25-2020.csv	06-05-2020.csv	08-16-2020.csv	10-27-2020.csv
01-04-2021.csv	03-26-2020.csv	06-06-2020.csv	08-17-2020.csv	10-28-2020.csv
01-05-2021.csv	03-27-2020.csv	06-07-2020.csv	08-18-2020.csv	10-29-2020.csv
01-06-2021.csv	03-28-2020.csv	06-08-2020.csv	08-19-2020.csv	10-30-2020.csv
01-07-2021.csv	03-29-2020.csv	06-09-2020.csv	08-20-2020.csv	10-31-2020.csv
01-08-2021.csv	03-30-2020.csv	06-10-2020.csv	08-21-2020.csv	11-01-2020.csv
01-09-2021.csv	03-31-2020.csv	06-11-2020.csv	08-22-2020.csv	11-02-2020.csv
01-10-2021.csv	04-01-2020.csv	06-12-2020.csv	08-23-2020.csv	11-03-2020.csv
01-11-2021.csv	04-02-2020.csv	06-13-2020.csv	08-24-2020.csv	11-04-2020.csv
01-22-2020.csv	04-03-2020.csv	06-14-2020.csv	08-25-2020.csv	11-05-2020.csv
01-23-2020.csv	04-04-2020.csv	06-15-2020.csv	08-26-2020.csv	11-06-2020.csv
01-24-2020.csv	04-05-2020

Com heu pogut observer en l'execució anterior, hi ha tot un seguit d'arxius, en concret un per cada dia, amb les dades COVID-19 corresponents al dia en qüestió.

El nostre objectiu ara és ajuntar les dades de tots aquests arxius i crear una única taula en la que figurin les dades de tots els dies. Aquesta tasca prèvia de conversió del format de les dades és molt habitual ja que les dades en brut acostumen a venir d'origens diversos o han estat obtenides per investigadors diversos i s'han d'agruapar.

Per crear i manipular taules de dades farem servir la biblioteca de funcions **pandas**. Si voleu fer un cop d'ull a les diferents possibilitats que ofereix podeu consultar la plana https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

El codi que ve a continuació llegeix cadascún dels arxius diaris i els va agrupant en una única taula i afegeix a cada línea de la taula la data del dia en que s'han recollit les dades. Les dades originals estan en format `csv` que és un format tabular i que els programes de full de càlcul com Microsoft Excel també el poden fer servir. La taula en si es guarda en una variable que anomenem df i és per això que no apareix cap resultat al executar el paràgraf següent.

In [7]:
path = r'COVID-19/csse_covid_19_data/csse_covid_19_daily_reports' # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))

df = pd.concat((pd.read_csv(f).assign(Date=lambda df: os.path.basename(f)[:-4]) for f in all_files))
df.Date = pd.to_datetime(df.Date, format='%m-%d-%Y')

Si volem veure la taula que acabem de crear només cal que li diguem a l'ordinador que ens la mostri escribint el nom de la variable que la conté, és a dir `df`. A l'executar el paràgraf següent, se'ns mostrarà la taula amb les dades. Observeu que al final ens indica el número d'observacions (més d'un mil·lió). Donat que mostrar el mil·lió d'observacions per pantalla seria inviable, el que es fa és mostrar uns punts suspensius a la meitat de la taula per indicar que hi ha més observacions entremig.

Donat que el número de columnes supera l'amplada de la pàgina, podeu lliscar horitzontalment el paràgraf amb el ratolí per observar les columnes ocultes

In [8]:
df

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Date,Incidence_Rate,Case-Fatality_Ratio,Province/State,Country/Region,Last Update,Latitude,Longitude,Incident_Rate,Case_Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-05-23 02:32:42,34.223334,-82.461707,36.0,0.0,0.0,36.0,"Abbeville, South Carolina, US",2020-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22001.0,Acadia,Louisiana,US,2020-05-23 02:32:42,30.295065,-92.414197,289.0,15.0,0.0,274.0,"Acadia, Louisiana, US",2020-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,51001.0,Accomack,Virginia,US,2020-05-23 02:32:42,37.767072,-75.632346,711.0,11.0,0.0,700.0,"Accomack, Virginia, US",2020-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16001.0,Ada,Idaho,US,2020-05-23 02:32:42,43.452658,-116.241552,794.0,23.0,0.0,771.0,"Ada, Idaho, US",2020-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19001.0,Adair,Iowa,US,2020-05-23 02:32:42,41.330756,-94.471059,6.0,0.0,0.0,6.0,"Adair, Iowa, US",2020-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3714,NaN,NaN,NaN,Vietnam,2020-06-11 03:33:41,14.058324,108.277199,332.0,0.0,320.0,12.0,Vietnam,2020-06-10,0.341077,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3715,NaN,NaN,NaN,West Bank and Gaza,2020-06-11 03:33:41,31.952200,35.233200,485.0,3.0,410.0,72.0,West Bank and Gaza,2020-06-10,9.507164,0.618557,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3716,NaN,NaN,NaN,Yemen,2020-06-11 03:33:41,15.552727,48.516388,560.0,129.0,23.0,408.0,Yemen,2020-06-10,1.877559,23.035714,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3717,NaN,NaN,NaN,Zambia,2020-06-11 03:33:41,-13.133897,27.849332,1200.0,10.0,912.0,278.0,Zambia,2020-06-10,6.527431,0.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Observeu que la taula que hem obtingut té una sèrie de mancances:

1. Tè variables repetides, per exemple Latitud i Longuitut, Incidència, etc.
2. Hi ha valors no disponibles que s'indiquen amb l'expressió `NaN` (Non available Number).
3. Hi ha columnes que no ens interessen com poden ser FIPS i Admin2

Caldrà doncs netejar i polir les ades abans de poder analitzar-les. Aquesta també és una tasca habitual quan treballem amb dades de recerca.

## Com netejem i polim les dades?

Primer unifiquem les variables repetides i després eliminem les repeticions. Les variables repetides apareixen perquè els criteris de recollida de dades han anat variant a mesura que ha anat avançant la pandèmia i el propi projecte de recollida de l'universitat John Hopkins.

In [9]:
df['Country_Region']=np.where(pd.isnull(df.Country_Region),df['Country/Region'],df['Country_Region'])
df['Province_State']=np.where(pd.isnull(df['Province_State']),df['Province/State'],df['Province_State'])

df['Latitude']=np.where(pd.isnull(df['Latitude']),df['Lat'],df['Latitude'])
df['Longitude']=np.where(pd.isnull(df['Longitude']),df['Long_'],df['Longitude'])

df['Case_Fatality_Ratio']=np.where(pd.isnull(df['Case_Fatality_Ratio']),df['Case-Fatality_Ratio'],df['Case_Fatality_Ratio'])
df['Case_Fatality_Ratio']=np.where(df['Deaths']==0,0,df['Case_Fatality_Ratio'])

df['Incidence_Rate']=np.where(pd.isnull(df['Incidence_Rate']),df['Incident_Rate'],df['Incidence_Rate'])

df['Last_Update']=np.where(pd.isnull(df['Last_Update']),df['Last Update'],df['Last_Update'])


df.drop(columns=['FIPS','Admin2','Country/Region','Province/State','Lat','Long_','Case-Fatality_Ratio','Incident_Rate','Last Update'], axis='columns',inplace=True)

Ara creem una nova columna que denominem Population que ens indica la població total de l'àrea geogràfica considerada donat que, tot hi haver utilitzat aquesta variable per calcular l'incidència per 100000 habitants, no ens la mostren en les dades originals.

In [10]:
df=df.assign(Population=lambda df: (df.Confirmed/df.Incidence_Rate)*100000)

Si tornem a demanar el contingut de la variable "df" veurem que la taula que tenim ara ja no té els errors de la taula original.

In [11]:
df

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,Combined_Key,Date,Incidence_Rate,Latitude,Longitude,Case_Fatality_Ratio,Population
0,South Carolina,US,2020-05-23 02:32:42,36.0,0.0,0.0,36.0,"Abbeville, South Carolina, US",2020-05-22,NaN,34.223334,-82.461707,0.000000,NaN
1,Louisiana,US,2020-05-23 02:32:42,289.0,15.0,0.0,274.0,"Acadia, Louisiana, US",2020-05-22,NaN,30.295065,-92.414197,NaN,NaN
2,Virginia,US,2020-05-23 02:32:42,711.0,11.0,0.0,700.0,"Accomack, Virginia, US",2020-05-22,NaN,37.767072,-75.632346,NaN,NaN
3,Idaho,US,2020-05-23 02:32:42,794.0,23.0,0.0,771.0,"Ada, Idaho, US",2020-05-22,NaN,43.452658,-116.241552,NaN,NaN
4,Iowa,US,2020-05-23 02:32:42,6.0,0.0,0.0,6.0,"Adair, Iowa, US",2020-05-22,NaN,41.330756,-94.471059,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3714,NaN,Vietnam,2020-06-11 03:33:41,332.0,0.0,320.0,12.0,Vietnam,2020-06-10,0.341077,14.058324,108.277199,0.000000,97338583.0
3715,NaN,West Bank and Gaza,2020-06-11 03:33:41,485.0,3.0,410.0,72.0,West Bank and Gaza,2020-06-10,9.507164,31.952200,35.233200,0.618557,5101416.0
3716,NaN,Yemen,2020-06-11 03:33:41,560.0,129.0,23.0,408.0,Yemen,2020-06-10,1.877559,15.552727,48.516388,23.035714,29825968.0
3717,NaN,Zambia,2020-06-11 03:33:41,1200.0,10.0,912.0,278.0,Zambia,2020-06-10,6.527431,-13.133897,27.849332,0.833333,18383956.0


Tot hi així, encara podem trobar files que no tenen dades Poblacionals, sobre tot en les dades de principis de 2020.

In [ ]:
df[pd.isnull(df.Population)].sort_values(['Date']).tail(100)

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,Combined_Key,Case_Fatality_Ratio,Date,Incidence_Rate,Latitude,Longitude,Population
1216,Hawaii,US,2020-12-21 13:27:30,286.0,2.0,0.0,284.0,"Out of HI, Hawaii, US",0.699301,2021-01-10,NaN,NaN,NaN,NaN
1210,Grand Princess,US,2020-08-04 02:27:56,103.0,3.0,0.0,100.0,"Grand Princess, US",2.912621,2021-01-10,NaN,NaN,NaN,NaN
1260,Idaho,US,2021-01-11 05:21:50,0.0,0.0,0.0,0.0,"Unassigned, Idaho, US",0.000000,2021-01-10,NaN,NaN,NaN,NaN
1158,Georgia,US,2021-01-11 05:21:50,27466.0,361.0,0.0,27105.0,"Out of GA, Georgia, US",1.314352,2021-01-10,NaN,NaN,NaN,NaN
1043,Florida,US,2021-01-11 05:21:50,3170.0,0.0,0.0,3170.0,"Unassigned, Florida, US",0.000000,2021-01-10,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,Alaska,US,2021-01-12 05:22:15,211.0,0.0,0.0,211.0,"Unassigned, Alaska, US",0.000000,2021-01-11,NaN,NaN,NaN,NaN
723,Alaska,US,2021-01-12 05:22:15,0.0,0.0,0.0,0.0,"Bristol Bay, Alaska, US",0.000000,2021-01-11,0.0,58.74514,-156.701064,NaN
702,Alabama,US,2020-12-21 13:27:30,0.0,0.0,0.0,0.0,"Out of AL, Alabama, US",0.000000,2021-01-11,NaN,NaN,NaN,NaN
1354,Illinois,US,2021-01-12 05:22:15,95.0,0.0,0.0,95.0,"Unassigned, Illinois, US",0.000000,2021-01-11,NaN,NaN,NaN,NaN


## Com analitzem les dades

Un cop tenim la taula endreçada en la variable `df`, podem transformarla de diferses maneres, per exemple, seleccionant només les dades que fan referència a Catalunya.

Ho farem referenciant la taula df i escribint la condició dins dels corchets, en aquest cas la condició és que la columna df.Combined_Key sigui igual a 'Catalona, Spain'

In [9]:
df[df.Combined_Key=='Catalonia, Spain']

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,Combined_Key,Date,Incidence_Rate,Latitude,Longitude,Case_Fatality_Ratio,Population
3045,Catalonia,Spain,2020-05-23 02:32:42,57036.0,6656.0,0.0,50380.0,"Catalonia, Spain",2020-05-22,NaN,41.5912,1.5209,NaN,NaN
588,Catalonia,Spain,2020-10-12 04:23:45,154354.0,5875.0,26203.0,122276.0,"Catalonia, Spain",2020-10-11,2039.984241,41.5912,1.5209,3.806186,7.566431e+06
3038,Catalonia,Spain,2020-05-22 02:36:51,55888.0,6021.0,0.0,49867.0,"Catalonia, Spain",2020-05-21,NaN,41.5912,1.5209,NaN,NaN
3327,Catalonia,Spain,2020-08-08 04:34:53,85970.0,5704.0,26203.0,54063.0,"Catalonia, Spain",2020-08-07,1136.202788,41.5912,1.5209,6.634873,7.566431e+06
589,Catalonia,Spain,2020-10-26 04:25:02,192035.0,5960.0,26203.0,159872.0,"Catalonia, Spain",2020-10-25,2537.986536,41.5912,1.5209,3.103601,7.566431e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3316,Catalonia,Spain,2020-07-25 04:47:39,72919.0,5680.0,26203.0,41036.0,"Catalonia, Spain",2020-07-24,963.717240,41.5912,1.5209,7.789465,7.566431e+06
588,Catalonia,Spain,2020-09-25 04:23:00,134825.0,5824.0,26203.0,102798.0,"Catalonia, Spain",2020-09-24,1781.883691,41.5912,1.5209,4.319674,7.566431e+06
603,Catalonia,Spain,2020-12-14 05:26:27,317636.0,8330.0,26203.0,283103.0,"Catalonia, Spain",2020-12-13,4197.963346,41.5912,1.5209,2.622499,7.566431e+06
587,Catalonia,Spain,2020-08-21 04:27:41,97234.0,5722.0,26203.0,65309.0,"Catalonia, Spain",2020-08-20,1285.070861,41.5912,1.5209,5.884773,7.566431e+06


Si a part de seleccionar files volguéssim seleccionar columnes, les columnes dessitjades les afegiriem en un vector (ex: `['Province_State','Deaths']` també dins de corchets (per tant, dobles corchets en aquest cas)

In [10]:
df[df.Combined_Key=='Catalonia, Spain'][['Province_State','Deaths']]

,Province_State,Deaths
3045,Catalonia,6656.0
588,Catalonia,5875.0
3038,Catalonia,6021.0
3327,Catalonia,5704.0
589,Catalonia,5960.0
...,...,...
3316,Catalonia,5680.0
588,Catalonia,5824.0
603,Catalonia,8330.0
587,Catalonia,5722.0


Si volem mostrar les dades gràficament fem servir l'instrucció `alt.Chart` on cal indicar la taula (df) amb o sense condicions per filtrar les files, el tipus de gràfic (mark_line en l'exemple que ve a continuació) i quina és la variable x i y que, en l'exemple següent són Date per les x i Deaths per les y 

In [12]:
alt.Chart(df[df.Combined_Key=='Catalonia, Spain']).mark_line().encode(x='Date:T', y='Deaths')

alt.Chart(...)

Fixeu-vos com el gràfic mostra el canvi de criteri en el recompte de morts fet per la Generalitat de Catalunya el mes de juny de 2020

La temptació és forta i segur que voldreu comparar l'evolució de la mortalitat entre Madrid i Barcelona. En aquest cas el filtre de les files haurà de ser múltiple per poder seleccionar Madrid i Barcelona. Vegeu l'exemple següent amb les dues condicions separades per una barra vertical i on es postra la taula i a continuació el gràfic amb les dades agrupades per comunitat autònoma (en el gràfic s'indica amb el paràmetre "color")

In [14]:
df[(df.Combined_Key=='Catalonia, Spain') | (df.Combined_Key=='Madrid, Spain')]

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,Combined_Key,Date,Incidence_Rate,Latitude,Longitude,Case_Fatality_Ratio,Population
3045,Catalonia,Spain,2020-05-23 02:32:42,57036.0,6656.0,0.0,50380.0,"Catalonia, Spain",2020-05-22,NaN,41.5912,1.5209,NaN,NaN
3109,Madrid,Spain,2020-05-23 02:32:42,67425.0,8944.0,0.0,58481.0,"Madrid, Spain",2020-05-22,NaN,40.4168,-3.7038,NaN,NaN
588,Catalonia,Spain,2020-10-12 04:23:45,154354.0,5875.0,26203.0,122276.0,"Catalonia, Spain",2020-10-11,2039.984241,41.5912,1.5209,3.806186,7566431.0
593,Madrid,Spain,2020-10-12 04:23:45,261762.0,9739.0,40736.0,211287.0,"Madrid, Spain",2020-10-11,3941.220019,40.4168,-3.7038,3.720555,6641649.0
3038,Catalonia,Spain,2020-05-22 02:36:51,55888.0,6021.0,0.0,49867.0,"Catalonia, Spain",2020-05-21,NaN,41.5912,1.5209,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,Madrid,Spain,2020-12-14 05:26:27,361368.0,11537.0,40736.0,309095.0,"Madrid, Spain",2020-12-13,5440.937936,40.4168,-3.7038,3.192590,6641649.0
587,Catalonia,Spain,2020-08-21 04:27:41,97234.0,5722.0,26203.0,65309.0,"Catalonia, Spain",2020-08-20,1285.070861,41.5912,1.5209,5.884773,7566431.0
592,Madrid,Spain,2020-08-21 04:27:41,99529.0,8526.0,40736.0,50267.0,"Madrid, Spain",2020-08-20,1498.558566,40.4168,-3.7038,8.566347,6641649.0
3125,Catalonia,Spain,2020-06-11 03:33:41,59298.0,5587.0,26203.0,27508.0,"Catalonia, Spain",2020-06-10,783.698417,41.5912,1.5209,9.421903,7566431.0


In [13]:
alt.Chart(df[(df.Combined_Key=='Catalonia, Spain') | 
             (df.Combined_Key=='Madrid, Spain')]).mark_line().encode(x='Date:T',y='Deaths',color='Combined_Key')

alt.Chart(...)

Si volguéssim analitzar la mortalitat acumulada en lloc de les morts absolutes, l'hauríem de calcular prèviament.

In [14]:
df=df.assign(Mortality=lambda df: (df.Deaths/df.Population)*100000)


In [15]:
alt.Chart(df[(df.Combined_Key=='Catalonia, Spain') | 
             (df.Combined_Key=='Madrid, Spain')]).mark_line().encode(x='Date:T',y='Mortality',color='Combined_Key')

alt.Chart(...)

Fixeu-vos també que abans del maig de 2020, les dades disponibles son les acumulades per tota España i no hi ha informació per comunitats autònomes. Per això

In [19]:
print(df[df.Combined_Key=='Spain'].Date.min(), ' ', df[df.Combined_Key=='Spain'].Date.max())

2020-03-22 00:00:00   2020-05-13 00:00:00
